<a href="https://colab.research.google.com/github/adarsh415/gpt/blob/main/gpt_bigram_singleHead_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-22 17:40:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-07-22 17:40:58 (38.8 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [86]:
# read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [87]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [88]:
#let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [89]:
# here are all unique characters that occurs in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [90]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, returns sequence of intgers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes list of integers, outputs string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [91]:
import torch
from torch import nn
import torch.nn.functional as F

In [92]:
batch_size=32 # how many independent sequences will process in parallel
block_size=8 # what is the maximum context length of predictions?
n_embed = 32
lr_rate = 1e-3
eval_interval = 500
eval_iters = 200
max_iters = 5000

In [93]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [94]:
#Let's split dataset into train and validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [95]:
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [96]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context is: {context} target is: {target}")

when context is: tensor([18]) target is: 47
when context is: tensor([18, 47]) target is: 56
when context is: tensor([18, 47, 56]) target is: 57
when context is: tensor([18, 47, 56, 57]) target is: 58
when context is: tensor([18, 47, 56, 57, 58]) target is: 1
when context is: tensor([18, 47, 56, 57, 58,  1]) target is: 15
when context is: tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
when context is: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


In [97]:
torch.manual_seed(1023)
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when context is: {context.tolist()} target is: {target}")


inputs
torch.Size([32, 8])
tensor([[53,  1, 40, 43,  1, 58, 39, 50],
        [ 1, 42, 53,  1, 52, 53, 58,  1],
        [47, 58,  1, 47, 57,  1, 52, 53],
        [25, 13, 30, 19, 13, 30, 17, 32],
        [43,  1, 39, 58,  1, 58, 46, 63],
        [46, 47, 57,  1, 46, 53, 50, 63],
        [ 0, 26, 53, 61,  1, 57, 61, 39],
        [ 1, 52, 39, 51, 43,  1, 40, 43],
        [ 0, 25, 17, 26, 17, 26, 21, 33],
        [13, 16, 37,  1, 19, 30, 17, 37],
        [ 1, 59, 54,  1, 58, 46, 43,  1],
        [45, 46, 52, 43, 57, 57,  6,  1],
        [53, 10,  0, 61, 46, 63,  6,  1],
        [48, 53, 47, 52,  5, 42,  0, 14],
        [42,  8,  0,  0, 19, 24, 27, 33],
        [46, 39, 58,  1, 24, 43, 61, 47],
        [43, 56, 43,  2,  0,  0, 14, 33],
        [35, 43, 50, 41, 53, 51, 43,  6],
        [26, 34, 27, 24, 21, 27, 10,  0],
        [57,  1, 40, 50, 47, 52, 42,  1],
        [58, 46, 43, 56,  1, 57, 58, 39],
        [51, 39, 50, 50,  1, 39, 50, 43],
        [43, 58, 41, 46,  1, 57, 46, 56],
       

In [98]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute the attention scores ('affinities)
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) ---> (B, T, T) , used scaled attention here
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) , this is part of decoder so does not communicate with past
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    # perform the weighted aggregation of values
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
    return out

In [99]:
torch.manual_seed(1033)

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super(BigramLanguageModel, self).__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.sa_head = Head(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, target=None):
    B, T = idx.shape

    # idx and targets are both (B,T) tensors of integer
    token_embed = self.token_embedding_table(idx) # (B,T,C) where C is n_embed
    pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
    X = token_embed + pos_embed # (B, T, C)
    X = self.sa_head(X) # apply one head of self-attention (B, T, C)
    logits = self.lm_head(X) # (B, T, Vocab_size)

    if target is not None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(-1) # or target.view(B*T)
      loss = F.cross_entropy(logits, target)

    else:
      loss = None

    return logits, loss

  def generate(self, idx, max_new_char):
    #idx is (B, T) array of indices in current context
    for _ in range(max_new_char):
      # print(idx.shape)
      # print(idx)
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # print(idx_cond)
      # get the predictions
      logits, loss = self(idx_cond)
      # Focus only on last time step
      logits = logits[:, -1, :] # becomes [B, C]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      # append sampled index to running index
      idx = torch.cat((idx, idx_next), dim=1) # (b , T+1)

    return idx

m = BigramLanguageModel()
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# start_token = torch.zeros((1,1), dtype=torch.long)
# print(decode(m.generate(start_token, 100)[0].tolist()))

In [100]:
# create Optimizer
optimizer = torch.optim.Adam(m.parameters(), lr = lr_rate)

In [101]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      xb, yb = get_batch(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [102]:
for steps in range(max_iters):

  if steps % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.1883, val loss 4.1867
step 500: train loss 2.6732, val loss 2.6848
step 1000: train loss 2.5387, val loss 2.5329
step 1500: train loss 2.4780, val loss 2.4882
step 2000: train loss 2.4472, val loss 2.4581
step 2500: train loss 2.4245, val loss 2.4410
step 3000: train loss 2.4110, val loss 2.4271
step 3500: train loss 2.4087, val loss 2.4207
step 4000: train loss 2.3954, val loss 2.4082
step 4500: train loss 2.3833, val loss 2.4001
2.3512332439422607


In [103]:
start_token = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(start_token, 1000)[0].tolist()))


Firsourd manat.
Cin ed tex'ctigese om hemeyove then yoru Ad lt st kit
fou satnean sif, tllold or
r,e bre, thet is m'ss hect lthe my ind ove he scade ned oligucer cre wollo cer vend, hald herp cinmak foouengo fenert-olis of ed nwely spal 'mt'dongwh
Acasingthe hat utalli, me:
Ath I WARowus ascage ld nde
As oten,
Councare lomeler mor deras ys
rey gf wine,
'Bt llepf, habere:
Wher tri'encat thout bollel com it orndimul chit werberowsus lis hin wamer outo ow, chemy cker-'ser, blary mene Rnout yorear 'bando shayon ul avis fovile me bep-en hat oincexwccour lod ured bes, yavewe hen te
wat hingoange sy:
Thearn cais, fay, sme indyorfomandent-hanad ciadatenshe ayomont pan os stfand st.
Ne Ined.

Whed se hey mere mif, our beme ne hat pis ond.

ON!


CABaty cegur ve ors as? ige wopr Man kin
See dolo wtho wat upersster
Whes Pe
PA-tievens yy laewedemins din's st hintoru, ble mond hat dhe
R GCAISheit
Thowndourtoh andr ay moduciostt to ame tinge nor pay blud ise cos te!

MAs wik wandold, sust?
Nse nder

# The Mathematical trick in self-attention

In [17]:
# consider the following toy example

torch.manual_seed(1033)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# we want x[b,t] = mean_{i <= t} x[b, i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    prev = x[b,:t+1] # (T, C)
    xbow[b,t] = torch.mean(prev, 0) # mean over T

In [30]:
# Approve 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [33]:
# Version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

In [36]:
# Version 4 self-attention
torch.manual_seed(1033)
B,T,C = 4, 8, 32 # Batch, timesteps, channels
x = torch.randn(B,T,C)
# single head attention
head_size = 16
key = torch.nn.Linear(C,head_size, bias = False)
query = torch.nn.Linear(C, head_size, bias = False)
value = torch.nn.Linear(C, head_size, bias = False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
#v = key(x) # (B, T, head_size)

wei = q @ k.transpose(-2,-1) # (B,T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf')) # this line prevents tokens to see future token in sequence and should be used in decoder part
# we can comments above line in encoder part so that token can interact with each other.
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [38]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7859, 0.2141, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9380, 0.0449, 0.0171, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1615, 0.0673, 0.0268, 0.7443, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1607, 0.1225, 0.5069, 0.0831, 0.1268, 0.0000, 0.0000, 0.0000],
        [0.1984, 0.0583, 0.0603, 0.4056, 0.0650, 0.2123, 0.0000, 0.0000],
        [0.0640, 0.1126, 0.0121, 0.1895, 0.0254, 0.3290, 0.2674, 0.0000],
        [0.5500, 0.0317, 0.0150, 0.3041, 0.0131, 0.0272, 0.0273, 0.0316]],
       grad_fn=<SelectBackward0>)

In [19]:
x[0]

tensor([[-0.4115,  1.2941],
        [-0.1356,  0.5413],
        [ 0.9311, -0.5061],
        [ 0.7235,  0.9506],
        [ 1.3993,  0.4271],
        [-0.9396,  0.1607],
        [-0.1828,  1.3827],
        [-1.3406, -0.2682]])

In [20]:
xbow[0]

tensor([[-0.4115,  1.2941],
        [-0.2735,  0.9177],
        [ 0.1280,  0.4431],
        [ 0.2769,  0.5700],
        [ 0.5013,  0.5414],
        [ 0.2612,  0.4780],
        [ 0.1978,  0.6072],
        [ 0.0055,  0.4978]])

In [23]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [104]:

torch.manual_seed(1033)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[8., 5.],
        [0., 7.],
        [7., 7.]])
c=
tensor([[8.0000, 5.0000],
        [4.0000, 6.0000],
        [5.0000, 6.3333]])
